In [1]:
# import files

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import re
from datetime import date, timedelta
import pandas as pd

import time

In [2]:
# Global variables
search_field_term_job = "data scientist"
search_field_term_location = "New York, NY"


In [3]:
# define driver
driver = webdriver.Chrome()

# open url
driver.get("https://www.simplyhired.com/")

# wait for the page to load
driver.implicitly_wait(5)


# find and click on search field location
search_field = driver.find_element(By.XPATH, '//*[@id="field-:R5bakt9fbqm:"]')
search_field.click()

# clear search field
#search_field.clear()
search_field.send_keys(Keys.CONTROL + "a")
search_field.send_keys(Keys.DELETE)

# enter search term job
search_field.send_keys(search_field_term_location)




# find and click on search field job
search_field = driver.find_element(By.XPATH, '//*[@id="field-:R3bakt9fbqm:"]')
search_field.click()

# wait for the page to load
driver.implicitly_wait(5)

# enter search term job
search_field.send_keys(search_field_term_job)

element = driver.switch_to.active_element
element.send_keys(Keys.ESCAPE)

# wait for the page to load
driver.implicitly_wait(5)



# submit form
# search_field.submit()
search_field.send_keys(Keys.ENTER)


# set for past 24 hours
driver.find_element(By.XPATH, '//*[@id="menu-button-:rl:"]').click()
time.sleep(2)
driver.find_elements(By.XPATH, '//*[@id="menu-list-:rl:"]//button')[1].click()



In [4]:
def getElemt(xpath, wait):
    wait.until(EC.visibility_of_element_located((By.XPATH, xpath)))
    return driver.find_element(By.XPATH, xpath)

In [5]:
def pullJobData():

    time.sleep(3)
    # get number of jobs
    webElementJobList = driver.find_element(By.XPATH, '//*[@id="job-list"]')
    num_rows = len(webElementJobList.find_elements(By.XPATH, '//li[@class="css-0"]'))
    wait = WebDriverWait(driver, timeout=5)


    # if link not found, skip to next job
    skip = False

    # list of jobs
    job_list = []

    for i in range(1, num_rows+1):

        job_desc = {
                    "jobTitle" : '',
                    'company':'',
                    'location':'',
                    'salary':'',
                    'date pulled': date.today(),
                    'link':'',
                    'description':'',
                    'qualifications':''
                
                    }

        # Get job title
        jobTitleXpath = f'//li[@class="css-0"][{i}]//a'
        for tries in range(2):
            try:
                job_desc["jobTitle"] = getElemt(jobTitleXpath, wait = wait).text
                break
            except Exception as e:
                print('job title not found')

        # Get link
        linkXpath =  f'//li[@class="css-0"][{i}]//a'
        for tries in range(2):
            try:
                job_desc['link'] = getElemt(linkXpath, wait = wait).get_attribute('href')

                break
            except Exception as e:
                print('link not found')


        # click on job
        linkXpath =  f'//li[@class="css-0"][{i}]//a'
        for tries in range(2):
            try:
                getElemt(linkXpath, wait = wait).click()
                break
            except:
                print('link not clickable')

                # skip to next job
                skip = True

        if skip:
            continue

        time.sleep(3)
        # Get description
        descriptionXpath = '//*[@data-testid="viewJobBodyJobFullDescriptionContent"]'
        for tries in range(2):
            try:
                job_desc['description'] = getElemt(descriptionXpath, wait = wait).text
                break
            except:
                print('description not found')

        # Get company and location
        xpath = '//*[@data-testid="viewJobHeadingContainer"]//span[@data-testid="detailText"]'
        for tries in range(2):
            try:
                wait.until(EC.visibility_of_element_located((By.XPATH, xpath)))
                job_desc['company'], job_desc['location']  = [ele.text for ele in driver.\
                        find_elements(By.XPATH, xpath)]
                break
            except:
                print('company and location not found')

        # Get salary
        salaryXpath = '//*[@data-testid="viewJobBodyJobCompensation"]'
        for tries in range(2):
            try:
                job_desc['salary']  = getElemt(salaryXpath, wait = wait).text
                break
            except:
                print('Salary not found')
        

        # Get qualification
        xpath = '//*[@id="__next"]/div/main/div/div[2]/div/div/div[2]/div/aside/div/div[1]/div/div[3]/div/div/ul//li'
        for tries in range(2):
            try:
                qualifications = [detail.text for detail in driver.find_elements(By.XPATH, xpath)]
                job_desc['qualifications'] = "||".join(qualifications)
                break
            except:
                print('qualifications not found')
                
        job_list.append(job_desc)
    
    return job_list

In [6]:
jobs = []

for _ in range(2):
    print(_)
    jobs.extend(pullJobData())

    try:
        next_page = driver.find_element(By.XPATH, '//*[@id="__next"]/div/main//nav//span/following-sibling::a').click()
        next_page.click()
    except:
        print('No more pages')
        break

    # let page load(driver.implicitly_wait does not work
    time.sleep(3)




0
No more pages


In [7]:
jobs = pd.DataFrame(jobs) #, columns = ['jobTitle', 'company', 'location', 'salary', 'date posted', 'link', 'describtion', 'qualifications'])
jobs.head()

,jobTitle,company,location,salary,date pulled,link,description,qualifications
0,Machine Learning,North Talent Group,"New York, NY",$50 - $53 an hour,2024-02-08,https://www.simplyhired.com/job/GhSLSxquUIrUqk...,Must be local to New York\nDesign and develop ...,Scikit-learn||7 years||PyTorch||NumPy||System ...
1,Software Engineer - Data Quality,Second Spectrum,"New York, NY","$125,000 - $155,000 a year",2024-02-08,https://www.simplyhired.com/job/G5OXpL3tOiDhQr...,Software Engineer - Data Quality\nat Second Sp...,Rust||Bachelor of Science||DevOps||Data analys...
2,GenAI Testing/NLP Automation Tester,Ccube,"New York, NY",$70 - $75 an hour,2024-02-08,https://www.simplyhired.com/job/6UGio0gE7FVI1h...,GenAI Testing/NLP AI Tester\nLocation : NYC (N...,Big data||DevOps||SQL||Analysis skills||Natura...
3,Data Science Manager - Remote,Optum,"New York, NY","$101,200 - $184,000 a year",2024-02-08,https://www.simplyhired.com/job/896IetuFQONlIb...,Optum is a global organization that delivers c...,Azure||6 years||Master's degree||SQL||Machine ...
4,Real Estate Data Management & Strategic Initia...,JPMorgan Chase & Co,"New York, NY","$100,985 - $185,000 a year",2024-02-08,https://www.simplyhired.com/job/9N2A0lMGLxBN_8...,JOB DESCRIPTION\n\nJob Description:\nOur Asset...,7 years||Microsoft Excel||Management||Yardi||A...


In [8]:
jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   jobTitle        9 non-null      object
 1   company         9 non-null      object
 2   location        9 non-null      object
 3   salary          9 non-null      object
 4   date pulled     9 non-null      object
 5   link            9 non-null      object
 6   description     9 non-null      object
 7   qualifications  9 non-null      object
dtypes: object(8)
memory usage: 704.0+ bytes


In [9]:
jobs.to_csv(f'data/simplyHiredScrape_{date.today()}.csv', index = False )

In [10]:
driver.close()